In [1]:
import urllib.request as ur # libreria para manipular peticiones 
import re                   # libreria para manejar expresiones regulares

f = ur.urlopen('https://resultados.as.com/resultados/futbol/primera/jornada') # consulta a pagina web  
s = f.read().decode() # manipulacion de respuesta web 


In [2]:
# Método para eliminar tags html que no nos interesan con ER 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext

In [3]:
from pprint import pprint     # import para hacer prints boninitos 

#Obtener url de cada jornada 
urlJornadas = re.findall('<li>.*?cont-jornada" href="(.*?)".*?</li>',s,re.DOTALL)

In [4]:
import datetime # libreria para manejo de fecha 

now = datetime.datetime.now() #obtener fecha actual 
# Arreglo de meses que inicia en agosto como la temporada 
months = ['Ago','Sep','Oct','Nov','Dic','Ene','Feb','Mar','Abr','May','Jun','Jul'] 
day =  now.day # obtener el numero del dia actual 
month = (now.month + 4) % 12 # obtener mes actual y ajustarlo para que coincida con el orden de meses de la temporada

In [5]:
# print results 
def printResults():
    print("Resultados condensados")
    print("G:Ganados   P:Perdidos   E:Empatados   GF:Goles a favor   GC:Goles en contra")
    print("\nEquipo\t\tG\tP\tE\tGF\tGC")
    for x in tablaResultados:
        data = x
        if len(data) > 7:
            data = data + "\t" + str(tablaResultados.get(x)[0])
        else: 
            data = data + "\t\t" + str(tablaResultados.get(x)[0])
        data = data + "\t" + str(tablaResultados.get(x)[1])
        data = data + "\t" + str(tablaResultados.get(x)[2])
        data = data + "\t" + str(tablaResultados.get(x)[3])
        data = data + "\t" + str(tablaResultados.get(x)[4])
        print(data)

In [6]:
'''
Método goles a favor
'''
def aFavor(E,G):
    tablaResultados.get(E)[3] = tablaResultados.get(E)[3] + int(G)
    

In [7]:
'''
Método goles en contra
'''
def enContra(E,G):
    tablaResultados.get(E)[4] = tablaResultados.get(E)[4] + int(G)

In [8]:
'''
Método empatar
'''

def empatar(E1,E2,R1):
    tablaResultados.get(E1)[2] = tablaResultados.get(E1)[2] + 1
    tablaResultados.get(E2)[2] = tablaResultados.get(E2)[2] + 1
    aFavor(E1,R1)
    enContra(E1,R1)
    aFavor(E2,R1)
    enContra(E2,R1)

In [9]:
'''
Metodo ganar 
'''

def ganar(E,GF,GC): #E: equipo GF goles favor GC goles contra
    print("tabla resultadooos")
    print(tablaResultados)
    print("E")
    print(E)
    print("GF")
    print(GF)
    print("GC")
    print(GC)
    print('tablaResultados.get(E)[0]')
    print(tablaResultados.get(E))
    tablaResultados.get(E)[0] = tablaResultados.get(E)[0] + 1
    aFavor(E,GF)
    enContra(E,GC)
    
    

In [10]:
'''
Metodo perder
'''
def perder(E,GF,GC): #E: equipo GF goles favor GC goles contra
    tablaResultados.get(E)[1] = tablaResultados.get(E)[1] + 1
    aFavor(E,GF)
    enContra(E,GC)
    

In [11]:
'''
Lógica perder ganar o empatar
'''

def decideGanador(datos):
    print("Decide ganador\ndatos")
    print(datos)
    if int(datos[2]) == int(datos[3]): #empate
        empatar(datos[0],datos[4],datos[2])
    elif int(datos[2]) > int(datos[3]): # gana equipo1 
        ganar(datos[0],datos[2],datos[3])
        perder(datos[4],datos[3],datos[2])
    else: # gana equipo 2
        ganar(datos[4],datos[3],datos[2])
        perder(datos[0],datos[2],datos[3])

In [12]:
'''
Tabla que contiene los resultados 
Los datos tienen la siguiente estructura 
{'Equipo':['Ganados','Perdidos','Empatados','Goles Favor','Goles Contra'],...}
'''

tablaResultados = {}
def addResults(listaResultados):
    print("Add results\nlista resultados")
    print(listaResultados)
    for x in listaResultados:
        if(tablaResultados.get(x[0]) is None):
            tablaResultados.update({x[0]:[0,0,0,0,0]})
        if(tablaResultados.get(x[4]) is None):
            tablaResultados.update({x[4]:[0,0,0,0,0]})
        decideGanador(x)

In [13]:
'''
Métodos para lógica del pasado 
'''
# Para pasados 
def pasados():
    # Obtener Equipo1 GolesE1 GolesE2 Equipo2
    marcadorPasado = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="resultado">.*?((..).-.(..)).*?</div>.*?class="nombre-equipo".*?>(.*?)</span>',jornadai,re.DOTALL)
    print("Resultados: ")
    print("tablaResultados")
    addResults(marcadorPasado)
    print()
    for x in marcadorPasado:
        print(x[0] + x[1] +x[4])

def printPast(jornadaFecha):
    if jornadaFecha[0][2] == '':  
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizó del " + jornadaFecha[0][1] + " al "+ jornadaFecha[0][3] + " de " + jornadaFecha[0][4] )
        else: 
             print("La " + jornadaFecha[0][0] + " se realizó el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
    else:
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizó del " + jornadaFecha[0][1] + " de " + jornadaFecha[0][2] + " al "+ jornadaFecha[0][3] + " de " + jornadaFecha[0][4] )
        else: 
             print("La " + jornadaFecha[0][0] + " se realizó el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
    pasados()

    


In [14]:
'''
Métodos para lógica de actual 
'''

# Para actuales 
# Obtener Equipo1 GolesE1-GolesE2 GolesE1 GolesE2 Equipo2 para los que lo tengan 
def actuales():
    listaResultados = jornadai.split('<li class="list-resultado">')
    listaResultados.pop(0)

    jugadosActual = []
    futurosActual = []

    for x in range(10):
        marcadorActual = []
        marcadorActual = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="resultado">.*?((..).-.(..)).*?</div>.*?class="nombre-equipo".*?>(.*?)</span>',listaResultados[x],re.DOTALL)
        if marcadorActual[0][1] == '   -   ': 
            juegosFuturos = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="nombre-equipo".*?>(.*?)</span>.*?<span class="fecha">(.*?)</span>',listaResultados[x],re.DOTALL)
            futurosActual.append(juegosFuturos[0])
        else:
            jugadosActual.append(marcadorActual[0])

    print("Resultados partidos jugados:")        
    for x in jugadosActual: 
        print(x[0] + x[1] +x[4])
    
    print("\nPartidos restantes:")
    for x in futurosActual:
        print(x[0] + " - " + x[1] + " en la fecha y horario " + x[2])

        

def printCurrent(jornadaFecha):
    print("La " + jornadaFecha[0][0] + " se está realizando")
    actuales()


In [15]:
'''
Métodos para lógica del futuro
'''

# para los que no lo tengan, obtener Equipo1 Equipo2 FechaDeJuego

# Para futuros
# obtener Equipo1 Equipo2 FechaDeJuego
def futuros():
    juegosFuturos = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="nombre-equipo".*?>(.*?)</span>.*?<span class="fecha">(.*?)</span>',jornadai,re.DOTALL)
    print("Partidos pendientes:")
    for x in juegosFuturos:
        print(x[0] + " - " + x[1] + " en la fecha y horario " + x[2])

        
        
def printFuture(jornadaFecha): 
    if jornadaFecha[0][2] == '':
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizará del " + jornadaFecha[0][1] + " al "+ jornadaFecha[0][3] +" de " + jornadaFecha[0][4] )
        else: 
            print("La " + jornadaFecha[0][0] + " se realizará el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
            
    else: 
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizará del " + jornadaFecha[0][1] + " de " + jornadaFecha[0][2]+ " al "+ jornadaFecha[0][3] +" de " + jornadaFecha[0][4] )
        else: 
            print("La " + jornadaFecha[0][0] + " se realizará el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
    futuros()

In [16]:
'''Metodo para decidir punto en el tiempo'''

def pointInTime(jornadaFecha):
    if jornadaFecha[0][2] == '':  
        if months.index(jornadaFecha[0][4]) > month:
            printFuture(jornadaFecha)
        elif months.index(jornadaFecha[0][4]) < month:
            printPast(jornadaFecha)
        elif months.index(jornadaFecha[0][4]) == month:
            if int(jornadaFecha[0][1]) > day:
                printFuture(jornadaFecha)
            elif jornadaFecha[0][3] != '':
                if day <= int(jornadaFecha[0][3]) and  day >= int(jornadaFecha[0][1]):
                    printCurrent(jornadaFecha)
                else: 
                    printPast(jornadaFecha)
            elif int(jornadaFecha[0][1]) == day:
                printCurrent(jornadaFecha)
            else:
                printPast(jornadaFecha)
    else: 
        if months.index(jornadaFecha[0][2]) > month:
            printFuture(jornadaFecha)
        elif months.index(jornadaFecha[0][4]) < month:
            printPast(jornadaFecha)
        elif months.index(jornadaFecha[0][2]) < month: # estamos en mes 2 
            if jornadaFecha[0][3] < day:
                printPast(jornadaFecha)
            else:
                printCurrent(jornadaFecha)
        else: # Estamos en mes 1 
            if jornadaFecha[0][1] > day:
                printFuture(jornadaFecha)
            else:
                printCurrent(jornadaFecha)

In [17]:
#Abrir la jornada correspondiente para cada url 
for x in urlJornadas:
    f = ur.urlopen("https://resultados.as.com"+x)
    jornadai = f.read().decode()
    #print(s + "\n")
    #Obtener número de jornada y fecha 
    jornadaFecha = re.findall('<h2 class="tit-modulo">.*?(Jornada \d*).*?>(\d\d) (.*?)* *-* *(\d\d)* *(...)<.*?</h2>',jornadai,re.DOTALL)
    
    # Decidir si es pasado futuro o presente e imprimir datos
    pointInTime(jornadaFecha)

        
    print("\n\n\n")
    


La Jornada 1 se realizó del 17 al 20 de Ago
Resultados: 
tablaResultados
Add results
lista resultados
[('Girona', ' 0 - 0 ', ' 0', '0 ', 'Valladolid'), ('Betis', ' 0 - 3 ', ' 0', '3 ', 'Levante'), ('Celta', ' 1 - 1 ', ' 1', '1 ', 'Espanyol'), ('Villarreal', ' 1 - 2 ', ' 1', '2 ', 'R. Sociedad'), ('Barcelona', ' 3 - 0 ', ' 3', '0 ', 'Alavés'), ('Eibar', ' 1 - 2 ', ' 1', '2 ', 'Huesca'), ('Rayo', ' 1 - 4 ', ' 1', '4 ', 'Sevilla'), ('Real Madrid', ' 2 - 0 ', ' 2', '0 ', 'Getafe'), ('Valencia', ' 1 - 1 ', ' 1', '1 ', 'Atlético'), ('Athletic', ' 2 - 1 ', ' 2', '1 ', 'Leganés')]
Decide ganador
datos
('Girona', ' 0 - 0 ', ' 0', '0 ', 'Valladolid')
Decide ganador
datos
('Betis', ' 0 - 3 ', ' 0', '3 ', 'Levante')
tabla resultadooos
{'Girona': [0, 0, 1, 0, 0], 'Valladolid': [0, 0, 1, 0, 0], 'Betis': [0, 0, 0, 0, 0], 'Levante': [0, 0, 0, 0, 0]}
E
Levante
GF
3 
GC
 0
tablaResultados.get(E)[0]
[0, 0, 0, 0, 0]
Decide ganador
datos
('Celta', ' 1 - 1 ', ' 1', '1 ', 'Espanyol')
Decide ganador
datos
('V

La Jornada 3 se realizó del 31 al 24 de Oct
Resultados: 
tablaResultados
Add results
lista resultados
[('Getafe', ' 0 - 0 ', ' 0', '0 ', 'Valladolid'), ('Villarreal', ' 0 - 1 ', ' 0', '1 ', 'Girona'), ('Eibar', ' 2 - 1 ', ' 2', '1 ', 'R. Sociedad'), ('Celta', ' 2 - 0 ', ' 2', '0 ', 'Atlético'), ('Real Madrid', ' 4 - 1 ', ' 4', '1 ', 'Leganés'), ('Levante', ' 2 - 2 ', ' 2', '2 ', 'Valencia'), ('Alavés', ' 2 - 1 ', ' 2', '1 ', 'Espanyol'), ('Barcelona', ' 8 - 2 ', ' 8', '2 ', 'Huesca'), ('Betis', ' 1 - 0 ', ' 1', '0 ', 'Sevilla'), ('Rayo', ' 1 - 1 ', ' 1', '1 ', 'Athletic')]
Decide ganador
datos
('Getafe', ' 0 - 0 ', ' 0', '0 ', 'Valladolid')
Decide ganador
datos
('Villarreal', ' 0 - 1 ', ' 0', '1 ', 'Girona')
tabla resultadooos
{'Girona': [0, 1, 1, 1, 4], 'Valladolid': [0, 1, 2, 0, 1], 'Betis': [0, 1, 1, 0, 3], 'Levante': [1, 1, 0, 4, 2], 'Celta': [1, 0, 1, 3, 2], 'Espanyol': [1, 0, 1, 3, 1], 'Villarreal': [0, 1, 1, 1, 2], 'R. Sociedad': [1, 0, 1, 4, 3], 'Barcelona': [2, 0, 0, 4, 0], 'A

La Jornada 5 se realizó del 21 al 23 de Sep
Resultados: 
tablaResultados
Add results
lista resultados
[('Huesca', ' 0 - 1 ', ' 0', '1 ', 'R. Sociedad'), ('Rayo', ' 1 - 5 ', ' 1', '5 ', 'Alavés'), ('Eibar', ' 1 - 0 ', ' 1', '0 ', 'Leganés'), ('Celta', ' 3 - 3 ', ' 3', '3 ', 'Valladolid'), ('Getafe', ' 0 - 2 ', ' 0', '2 ', 'Atlético'), ('Real Madrid', ' 1 - 0 ', ' 1', '0 ', 'Espanyol'), ('Levante', ' 2 - 6 ', ' 2', '6 ', 'Sevilla'), ('Villarreal', ' 0 - 0 ', ' 0', '0 ', 'Valencia'), ('Betis', ' 2 - 2 ', ' 2', '2 ', 'Athletic'), ('Barcelona', ' 2 - 2 ', ' 2', '2 ', 'Girona')]
Decide ganador
datos
('Huesca', ' 0 - 1 ', ' 0', '1 ', 'R. Sociedad')
tabla resultadooos
{'Girona': [2, 1, 1, 5, 6], 'Valladolid': [0, 2, 2, 0, 2], 'Betis': [1, 1, 2, 1, 3], 'Levante': [1, 2, 1, 6, 5], 'Celta': [2, 1, 1, 7, 5], 'Espanyol': [2, 1, 1, 5, 3], 'Villarreal': [1, 2, 1, 2, 3], 'R. Sociedad': [1, 2, 1, 6, 7], 'Barcelona': [4, 0, 0, 14, 3], 'Alavés': [2, 1, 1, 3, 4], 'Eibar': [1, 2, 1, 4, 6], 'Huesca': [1, 2,

La Jornada 7 se realizó del 28 al 01 de Oct
Resultados: 
tablaResultados
Add results
lista resultados
[('Rayo', ' 2 - 2 ', ' 2', '2 ', 'Espanyol'), ('R. Sociedad', ' 0 - 1 ', ' 0', '1 ', 'Valencia'), ('Barcelona', ' 1 - 1 ', ' 1', '1 ', 'Athletic'), ('Eibar', ' 1 - 3 ', ' 1', '3 ', 'Sevilla'), ('Real Madrid', ' 0 - 0 ', ' 0', '0 ', 'Atlético'), ('Huesca', ' 1 - 1 ', ' 1', '1 ', 'Girona'), ('Villarreal', ' 0 - 1 ', ' 0', '1 ', 'Valladolid'), ('Levante', ' 2 - 1 ', ' 2', '1 ', 'Alavés'), ('Betis', ' 1 - 0 ', ' 1', '0 ', 'Leganés'), ('Celta', ' 1 - 1 ', ' 1', '1 ', 'Getafe')]
Decide ganador
datos
('Rayo', ' 2 - 2 ', ' 2', '2 ', 'Espanyol')
Decide ganador
datos
('R. Sociedad', ' 0 - 1 ', ' 0', '1 ', 'Valencia')
tabla resultadooos
{'Girona': [2, 2, 2, 7, 9], 'Valladolid': [1, 2, 3, 5, 6], 'Betis': [2, 1, 3, 4, 5], 'Levante': [1, 4, 1, 9, 13], 'Celta': [2, 1, 3, 11, 9], 'Espanyol': [3, 2, 2, 8, 6], 'Villarreal': [2, 2, 2, 5, 3], 'R. Sociedad': [2, 2, 2, 9, 9], 'Barcelona': [4, 1, 1, 17, 7], 

La Jornada 9 se realizó del 19 al 22 de Oct
Resultados: 
tablaResultados
Add results
lista resultados
[('Celta', ' 0 - 1 ', ' 0', '1 ', 'Alavés'), ('Real Madrid', ' 1 - 2 ', ' 1', '2 ', 'Levante'), ('Valencia', ' 1 - 1 ', ' 1', '1 ', 'Leganés'), ('Villarreal', ' 1 - 1 ', ' 1', '1 ', 'Atlético'), ('Barcelona', ' 4 - 2 ', ' 4', '2 ', 'Sevilla'), ('Rayo', ' 1 - 2 ', ' 1', '2 ', 'Getafe'), ('Eibar', ' 1 - 1 ', ' 1', '1 ', 'Athletic'), ('Huesca', ' 0 - 2 ', ' 0', '2 ', 'Espanyol'), ('Betis', ' 0 - 1 ', ' 0', '1 ', 'Valladolid'), ('R. Sociedad', ' 0 - 0 ', ' 0', '0 ', 'Girona')]
Decide ganador
datos
('Celta', ' 0 - 1 ', ' 0', '1 ', 'Alavés')
tabla resultadooos
{'Girona': [2, 3, 3, 10, 13], 'Valladolid': [3, 2, 3, 7, 6], 'Betis': [3, 2, 3, 5, 6], 'Levante': [3, 4, 1, 12, 14], 'Celta': [2, 2, 4, 13, 12], 'Espanyol': [4, 2, 2, 11, 7], 'Villarreal': [2, 4, 2, 6, 7], 'R. Sociedad': [3, 3, 2, 12, 11], 'Barcelona': [4, 1, 3, 19, 9], 'Alavés': [4, 2, 2, 11, 8], 'Eibar': [3, 4, 1, 9, 12], 'Huesca': [

...

La Jornada 19 se realizó del 11 al 14 de Ene
Resultados: 
tablaResultados
Add results
lista resultados
[('Rayo', ' 4 - 2 ', ' 4', '2 ', 'Celta'), ('Leganés', ' 1 - 0 ', ' 1', '0 ', 'Huesca'), ('Valencia', ' 1 - 1 ', ' 1', '1 ', 'Valladolid'), ('Girona', ' 1 - 1 ', ' 1', '1 ', 'Alavés'), ('Villarreal', ' 1 - 2 ', ' 1', '2 ', 'Getafe'), ('Atlético', ' 1 - 0 ', ' 1', '0 ', 'Levante'), ('Athletic', ' 2 - 0 ', ' 2', '0 ', 'Sevilla'), ('Barcelona', ' 3 - 0 ', ' 3', '0 ', 'Eibar'), ('Betis', ' 1 - 2 ', ' 1', '2 ', 'Real Madrid'), ('R. Sociedad', ' 3 - 2 ', ' 3', '2 ', 'Espanyol')]
Decide ganador
datos
('Rayo', ' 4 - 2 ', ' 4', '2 ', 'Celta')
tabla resultadooos
{'Girona': [5, 5, 8, 20, 22], 'Valladolid': [5, 7, 6, 16, 20], 'Betis': [7, 6, 5, 21, 21], 'Levante': [6, 7, 5, 30, 34], 'Celta': [5, 7, 6, 29, 28], 'Espanyol': [7, 8, 3, 19, 24], 'Villarreal': [3, 7, 8, 19, 23], 'R. Sociedad': [6, 8, 4, 20, 20], 'Barcelona': [12, 2, 4, 50, 20], 'Alavés': [9, 5, 4, 21, 18], 'Eibar': [5, 6, 7, 21, 25], '

La Jornada 21 se realizó del 26 al 28 de Ene
Resultados: 
tablaResultados
Add results
lista resultados
[('Sevilla', ' 5 - 0 ', ' 5', '0 ', 'Levante'), ('Atlético', ' 2 - 0 ', ' 2', '0 ', 'Getafe'), ('Leganés', ' 2 - 2 ', ' 2', '2 ', 'Eibar'), ('Valencia', ' 3 - 0 ', ' 3', '0 ', 'Villarreal'), ('Valladolid', ' 2 - 1 ', ' 2', '1 ', 'Celta'), ('Girona', ' 0 - 2 ', ' 0', '2 ', 'Barcelona'), ('R. Sociedad', ' 0 - 0 ', ' 0', '0 ', 'Huesca'), ('Athletic', ' 1 - 0 ', ' 1', '0 ', 'Betis'), ('Espanyol', ' 2 - 4 ', ' 2', '4 ', 'Real Madrid'), ('Alavés', ' 0 - 1 ', ' 0', '1 ', 'Rayo')]
Decide ganador
datos
('Sevilla', ' 5 - 0 ', ' 5', '0 ', 'Levante')
tabla resultadooos
{'Girona': [5, 6, 9, 23, 26], 'Valladolid': [5, 8, 7, 17, 23], 'Betis': [8, 7, 5, 25, 25], 'Levante': [7, 8, 5, 32, 35], 'Celta': [5, 9, 6, 32, 34], 'Espanyol': [7, 10, 3, 21, 30], 'Villarreal': [3, 8, 9, 21, 26], 'R. Sociedad': [7, 8, 5, 25, 24], 'Barcelona': [14, 2, 4, 56, 21], 'Alavés': [9, 6, 5, 22, 23], 'Eibar': [6, 7, 7, 24, 

La Jornada 23 se realizó del 08 al 11 de Feb
Resultados: 
tablaResultados
Add results
lista resultados
[('Valladolid', ' 0 - 0 ', ' 0', '0 ', 'Villarreal'), ('Getafe', ' 3 - 1 ', ' 3', '1 ', 'Celta'), ('Atlético', ' 1 - 3 ', ' 1', '3 ', 'Real Madrid'), ('Espanyol', ' 2 - 1 ', ' 2', '1 ', 'Rayo'), ('Girona', ' 0 - 2 ', ' 0', '2 ', 'Huesca'), ('Leganés', ' 3 - 0 ', ' 3', '0 ', 'Betis'), ('Valencia', ' 0 - 0 ', ' 0', '0 ', 'R. Sociedad'), ('Sevilla', ' 2 - 2 ', ' 2', '2 ', 'Eibar'), ('Athletic', ' 0 - 0 ', ' 0', '0 ', 'Barcelona'), ('Alavés', ' 2 - 0 ', ' 2', '0 ', 'Levante')]
Decide ganador
datos
('Valladolid', ' 0 - 0 ', ' 0', '0 ', 'Villarreal')
Decide ganador
datos
('Getafe', ' 3 - 1 ', ' 3', '1 ', 'Celta')
tabla resultadooos
{'Girona': [5, 8, 9, 23, 31], 'Valladolid': [6, 9, 8, 19, 28], 'Betis': [9, 8, 5, 26, 26], 'Levante': [7, 9, 6, 32, 40], 'Celta': [6, 10, 6, 34, 36], 'Espanyol': [7, 11, 4, 25, 36], 'Villarreal': [3, 9, 11, 23, 31], 'R. Sociedad': [8, 8, 6, 27, 25], 'Barcelona': 

La Jornada 25 se realizó del 22 al 25 de Feb
Resultados: 
tablaResultados
Add results
lista resultados
[('Espanyol', ' 1 - 1 ', ' 1', '1 ', 'Huesca'), ('Getafe', ' 2 - 1 ', ' 2', '1 ', 'Rayo'), ('Sevilla', ' 2 - 4 ', ' 2', '4 ', 'Barcelona'), ('Alavés', ' 0 - 0 ', ' 0', '0 ', 'Celta'), ('Athletic', ' 1 - 0 ', ' 1', '0 ', 'Eibar'), ('Leganés', ' 1 - 1 ', ' 1', '1 ', 'Valencia'), ('Atlético', ' 2 - 0 ', ' 2', '0 ', 'Villarreal'), ('Valladolid', ' 0 - 2 ', ' 0', '2 ', 'Betis'), ('Levante', ' 1 - 2 ', ' 1', '2 ', 'Real Madrid'), ('Girona', ' 0 - 0 ', ' 0', '0 ', 'R. Sociedad')]
Decide ganador
datos
('Espanyol', ' 1 - 1 ', ' 1', '1 ', 'Huesca')
Decide ganador
datos
('Getafe', ' 2 - 1 ', ' 2', '1 ', 'Rayo')
tabla resultadooos
{'Girona': [6, 9, 9, 25, 34], 'Valladolid': [6, 10, 8, 19, 29], 'Betis': [9, 9, 6, 27, 30], 'Levante': [8, 10, 6, 36, 43], 'Celta': [6, 12, 6, 36, 43], 'Espanyol': [8, 11, 6, 28, 38], 'Villarreal': [4, 9, 11, 26, 31], 'R. Sociedad': [9, 8, 7, 30, 25], 'Barcelona': [16, 

La Jornada 27 se realizará del 08 al 10 de Mar
Partidos pendientes:
Athletic - Espanyol en la fecha y horario V-08/03 21:00
Alavés - Eibar en la fecha y horario S-09/03 13:00
Atlético - Leganés en la fecha y horario S-09/03 16:15
Barcelona - Rayo en la fecha y horario S-09/03 18:30
Getafe - Huesca en la fecha y horario S-09/03 20:45
Celta - Betis en la fecha y horario D-10/03 12:00
Girona - Valencia en la fecha y horario D-10/03 16:15
Levante - Villarreal en la fecha y horario D-10/03 18:30
Sevilla - R. Sociedad en la fecha y horario D-10/03 18:30
Valladolid - Real Madrid en la fecha y horario D-10/03 20:45




La Jornada 28 se realizará del 15 al 17 de Mar
Partidos pendientes:
R. Sociedad - Levante en la fecha y horario V-15/03 21:00
Huesca - Alavés en la fecha y horario S-16/03 13:00
Real Madrid - Celta en la fecha y horario S-16/03 16:15
Athletic - Atlético en la fecha y horario S-16/03 18:30
Leganés - Girona en la fecha y horario S-16/03 20:45
Eibar - Valladolid en la fecha y horar

In [18]:
printResults()

Resultados condensados
G:Ganados   P:Perdidos   E:Empatados   GF:Goles a favor   GC:Goles en contra

Equipo		G	P	E	GF	GC
Girona		7	9	10	27	34
Valladolid	6	12	8	20	34
Betis		10	10	6	30	32
Levante		8	12	6	37	46
Celta		6	13	7	36	44
Espanyol	9	11	6	31	39
Villarreal	4	11	11	27	35
R. Sociedad	9	9	8	30	27
Barcelona	18	2	6	66	25
Alavés		11	8	7	27	29
Eibar		8	8	10	34	35
Huesca		5	14	7	26	42
Rayo		6	15	5	28	45
Sevilla		10	9	7	41	34
Real Madrid	15	8	3	43	31
Getafe		11	6	9	34	23
Valencia	7	4	15	27	21
Atlético	15	3	8	38	17
Athletic	7	7	12	25	30
Leganés		8	9	9	27	31


In [19]:
tablaRess = {'Equipo':['Ganados','Perdidos','Empatados','Goles Favor','Goles Contra'],
                    'E1':[0,1,2,3,4]}
pprint(tablaRess)
tablaRess.get('E1')[1] = tablaRess.get('E1')[1] + 1 
print(tablaRess.get('E1')[0] )
pprint(tablaRess)
print(tablaRess.get('1'))

tablaRess.update({'E3':[0,1,2,3,4]})
pprint(tablaRess)

{'E1': [0, 1, 2, 3, 4],
 'Equipo': ['Ganados', 'Perdidos', 'Empatados', 'Goles Favor', 'Goles Contra']}
0
{'E1': [0, 2, 2, 3, 4],
 'Equipo': ['Ganados', 'Perdidos', 'Empatados', 'Goles Favor', 'Goles Contra']}
None
{'E1': [0, 2, 2, 3, 4],
 'E3': [0, 1, 2, 3, 4],
 'Equipo': ['Ganados', 'Perdidos', 'Empatados', 'Goles Favor', 'Goles Contra']}
